* see: https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# removed OPenAI, using HF
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import hub

# removed OpenAI, using Cohere
from langchain.chat_models import ChatCohere

from langchain.schema.runnable import RunnablePassthrough

# private configurations
from config_private import COHERE_API_KEY

#### Loading the document

In [2]:
WEB_PAGES = [
    "https://luigi-saetta.medium.com/generative-ai-how-to-control-the-creativity-of-your-large-language-model-c7b0322b4c3d",
    "https://medium.com/the-modern-scientist/a-conversation-around-generative-ai-cca5c408ead4",
]


loader = WebBaseLoader(WEB_PAGES)

data = loader.load()

#### Splitting the document in chunks

In [3]:
CHUNK_SIZE = 512

text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=0)

splits = text_splitter.split_documents(data)

In [4]:
print(f"We have {len(splits)} splits...")

We have 42 splits...


In [5]:
splits[1]

Document(page_content='trained to generate text.In this article, I‚Äôll discuss some general characteristics of LLM for text generation. Especially those parameters that can be used, in the inference phase, to control the variety of possible answers to prompts. Then, I‚Äôll add some more details regarding Oracle OCI AI Generative Service, which is based on LLMs provided by Cohere.First of all, some basic concepts: how does an LLM generating text work, what is a prompt, and what does it mean that an LLM can be creative?An LLM for', metadata={'source': 'https://luigi-saetta.medium.com/generative-ai-how-to-control-the-creativity-of-your-large-language-model-c7b0322b4c3d', 'title': 'Generative AI: how to control the creativity of your Large Language Model | by Luigi Saetta | Oct, 2023 | Medium', 'description': 'At the heart of every Generative Service for Text, there is a Large Language Model (LLM), a model that has been trained, on a very large corpus of documents, to learn the structure 

In [6]:
# We have substitued OpenAI with HF
EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}


hf = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_NAME, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# using Chroma as Vector store
vectorstore = Chroma.from_documents(documents=splits, embedding=hf)

retriever = vectorstore.as_retriever()

#### Define the prompt structure

In [7]:
rag_prompt = hub.pull("rlm/rag-prompt")

#### Define the LLM

In [8]:
API_KEY = COHERE_API_KEY

llm = ChatCohere(cohere_api_key=API_KEY, temperature=0.5, verbose=True)

#### Define the (Lang)Chain

In [9]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | rag_prompt | llm

#### Process the question

In [10]:
QUESTION = """What is a Large Language Model (LLM)? Could you give some examples of what can we do with a LLM? 
Imagine that we’re talking to a eleven years old boy"""

response = rag_chain.invoke(QUESTION)

print(response.content)

A Large Language Model is a type of AI that is trained on massive amounts of data and designed to generate human-like responses to user requests. There are a plethora of ways to use them, for example, the Large Language Models can be used to generate text. This is one of many possible uses. 

Is this helpful?


#### Explore the vectore store

In [11]:
# Retrieve relevant splits for any question using similarity search.

# This is simply "top K" retrieval where we select documents based on embedding similarity to the query.

TOP_K = 5

docs = vectorstore.similarity_search(QUESTION, k=TOP_K)

len(docs)

5

In [12]:
docs

[Document(page_content='Model built by the company Cohere with a vast amount of information to help assist and engage in conversations with users. I am designed to generate human-like responses and assist with a variety of tasks.Okay, well you should try to be simpler. Imagine that we‚Äôre talking to a boy, let‚Äôs say, around eleven years old. What is a Large Language Model (LLM)? Could you give some examples of what can we do with a LLM?Sure, let‚Äôs simplify things with Zachary.You can think of a Large Language Model as a very', metadata={'description': 'Well, explaining AI is not easy, especially for non-technical people. But, nevertheless, we should try! So, I thought it was a good idea to speak directly to one of the protagonists of this field‚Ä¶', 'language': 'en', 'source': 'https://medium.com/the-modern-scientist/a-conversation-around-generative-ai-cca5c408ead4', 'title': 'A conversation around Generative AI | by Luigi Saetta | The Modern Scientist | Sep, 2023 | Medium'}),
 Do